In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Lab6").config("spark.driver.memory", "8g") \
.config("spark.executor.memory", "8g").config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
.getOrCreate()

In [3]:
data = spark.read.option("inferSchema", True).option("header", False).csv("covtype.data")
data.cache()

24/10/07 09:51:52 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


DataFrame[_c0: int, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: int, _c8: int, _c9: int, _c10: int, _c11: int, _c12: int, _c13: int, _c14: int, _c15: int, _c16: int, _c17: int, _c18: int, _c19: int, _c20: int, _c21: int, _c22: int, _c23: int, _c24: int, _c25: int, _c26: int, _c27: int, _c28: int, _c29: int, _c30: int, _c31: int, _c32: int, _c33: int, _c34: int, _c35: int, _c36: int, _c37: int, _c38: int, _c39: int, _c40: int, _c41: int, _c42: int, _c43: int, _c44: int, _c45: int, _c46: int, _c47: int, _c48: int, _c49: int, _c50: int, _c51: int, _c52: int, _c53: int, _c54: int]

In [4]:
colnames = ["Elevation", "Aspect", "Slope", "Horizontal_Distance_To_Hydrology", "Vertical_Distance_To_Hydrology", 
            "Horizontal_Distance_To_Roadways", "Hillshade_9am", "Hillshade_Noon", "Hillshade_3pm", 
            "Horizontal_Distance_To_Fire_Points"] + \
           [f"Wilderness_Area_{i}" for i in range(4)] + \
           [f"Soil_Type_{i}" for i in range(40)] + \
           ["Cover_Type"]

data = data.toDF(*colnames)

data.head()

Row(Elevation=2596, Aspect=51, Slope=3, Horizontal_Distance_To_Hydrology=258, Vertical_Distance_To_Hydrology=0, Horizontal_Distance_To_Roadways=510, Hillshade_9am=221, Hillshade_Noon=232, Hillshade_3pm=148, Horizontal_Distance_To_Fire_Points=6279, Wilderness_Area_0=1, Wilderness_Area_1=0, Wilderness_Area_2=0, Wilderness_Area_3=0, Soil_Type_0=0, Soil_Type_1=0, Soil_Type_2=0, Soil_Type_3=0, Soil_Type_4=0, Soil_Type_5=0, Soil_Type_6=0, Soil_Type_7=0, Soil_Type_8=0, Soil_Type_9=0, Soil_Type_10=0, Soil_Type_11=0, Soil_Type_12=0, Soil_Type_13=0, Soil_Type_14=0, Soil_Type_15=0, Soil_Type_16=0, Soil_Type_17=0, Soil_Type_18=0, Soil_Type_19=0, Soil_Type_20=0, Soil_Type_21=0, Soil_Type_22=0, Soil_Type_23=0, Soil_Type_24=0, Soil_Type_25=0, Soil_Type_26=0, Soil_Type_27=0, Soil_Type_28=1, Soil_Type_29=0, Soil_Type_30=0, Soil_Type_31=0, Soil_Type_32=0, Soil_Type_33=0, Soil_Type_34=0, Soil_Type_35=0, Soil_Type_36=0, Soil_Type_37=0, Soil_Type_38=0, Soil_Type_39=0, Cover_Type=5)

In [5]:
(train_data, test_data) = data.randomSplit([0.75, 0.25])

In [6]:
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col,udf
from pyspark.ml.feature import VectorAssembler

def unencode(data):
    cols = ['Wilderness_Area_' + str(i) for i in range(4)]
    assembler = VectorAssembler().setInputCols(cols).setOutputCol("wilderness")
    
    unhot_udf = udf(lambda v: v.toArray().tolist().index(1))
    
    data = assembler.transform(data).drop(*cols) \
    .withColumn("wilderness", unhot_udf(col("wilderness")).cast(IntegerType()))
    
    cols = ['Soil_Type_' + str(i) for i in range(40)]
    assembler = VectorAssembler().setInputCols(cols).setOutputCol("soil")
    
    data = assembler.transform(data).drop(*cols) \
    .withColumn("soil", unhot_udf(col("soil")).cast(IntegerType()))
    
    return data

In [7]:
unenc_train_data = unencode(train_data)
unenc_train_data.cache()
unenc_train_data.printSchema()

root
 |-- Elevation: integer (nullable = true)
 |-- Aspect: integer (nullable = true)
 |-- Slope: integer (nullable = true)
 |-- Horizontal_Distance_To_Hydrology: integer (nullable = true)
 |-- Vertical_Distance_To_Hydrology: integer (nullable = true)
 |-- Horizontal_Distance_To_Roadways: integer (nullable = true)
 |-- Hillshade_9am: integer (nullable = true)
 |-- Hillshade_Noon: integer (nullable = true)
 |-- Hillshade_3pm: integer (nullable = true)
 |-- Horizontal_Distance_To_Fire_Points: integer (nullable = true)
 |-- Cover_Type: integer (nullable = true)
 |-- wilderness: integer (nullable = true)
 |-- soil: integer (nullable = true)



In [8]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder,TrainValidationSplit
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

cols = unenc_train_data.columns
input_cols = [c for c in cols if c!='Cover_Type']

assembler = VectorAssembler(inputCols=input_cols, outputCol="features")

classifier = DecisionTreeClassifier(seed=1234, labelCol="Cover_Type",featuresCol="features",
predictionCol="pred")

pipeline = Pipeline(stages=[assembler, classifier])

paramGrid = ParamGridBuilder().addGrid(classifier.impurity, ["gini", "entropy"]) \
.addGrid(classifier.maxDepth, [1, 20]).addGrid(classifier.maxBins, [40, 300]) \
.addGrid(classifier.minInfoGain, [0.0, 0.05]).build()

multiclassEval = MulticlassClassificationEvaluator().setLabelCol("Cover_Type").setPredictionCol("pred"). \
setMetricName("accuracy")

trainValidationSplit = TrainValidationSplit(estimator=pipeline, estimatorParamMaps=paramGrid, trainRatio=0.8,
                                             evaluator=multiclassEval)


In [9]:
bestModel = trainValidationSplit.fit(unenc_train_data)

24/10/07 09:52:11 WARN DAGScheduler: Broadcasting large task binary with size 1079.1 KiB
24/10/07 09:52:12 WARN DAGScheduler: Broadcasting large task binary with size 1503.6 KiB
24/10/07 09:52:12 WARN DAGScheduler: Broadcasting large task binary with size 2.0 MiB
24/10/07 09:52:13 WARN DAGScheduler: Broadcasting large task binary with size 2.7 MiB
24/10/07 09:52:13 WARN DAGScheduler: Broadcasting large task binary with size 3.4 MiB
24/10/07 09:52:14 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
24/10/07 09:52:14 WARN DAGScheduler: Broadcasting large task binary with size 5.1 MiB
24/10/07 09:52:15 WARN DAGScheduler: Broadcasting large task binary with size 5.9 MiB
24/10/07 09:52:16 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
24/10/07 09:52:20 WARN DAGScheduler: Broadcasting large task binary with size 1080.7 KiB
24/10/07 09:52:21 WARN DAGScheduler: Broadcasting large task binary with size 1451.1 KiB
24/10/07 09:52:23 WARN DAGScheduler: Broad

In [14]:
from pyspark.mllib.evaluation import MulticlassMetrics

def evaluate_model(predictions, dataset_name):
    accuracy_evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="pred", metricName="accuracy")
    accuracy = accuracy_evaluator.evaluate(predictions)
    
    f1_evaluator = MulticlassClassificationEvaluator(labelCol="Cover_Type", predictionCol="pred", metricName="f1")
    f1_score = f1_evaluator.evaluate(predictions)
    
    prediction_and_labels = predictions.select("pred", "Cover_Type").rdd
    metrics = MulticlassMetrics(prediction_and_labels)
    
    confusion_matrix = metrics.confusionMatrix().toArray()
    
    print(f"Evaluation results for {dataset_name}:")
    print(f"Accuracy: {accuracy}")
    print(f"F1 Score: {f1_score}")
    print("Confusion Matrix:")
    print(confusion_matrix)
    print("\n")

In [15]:
trainpred = bestModel.transform(unenc_train_data)
trainpred.select("Cover_Type", "pred", "probability").show(10, truncate = False)

+----------+----+---------------------------------+
|Cover_Type|pred|probability                      |
+----------+----+---------------------------------+
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
|3         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]|
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]|
+----------+----+---------------------------------+
only showing top 10 rows



24/10/07 09:57:11 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


In [16]:
evaluate_model(trainpred, "Training Data")

24/10/07 09:57:12 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:12 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
24/10/07 09:57:13 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:13 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:13 ERROR Executor: Exception in task 9.0 in stage 489.0 (TID 7740)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  

	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$4(Executor.scala:620)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally(SparkErrorUtils.scala:64)
	at org.apache.spark.util.SparkErrorUtils.tryWithSafeFinally$(SparkErrorUtils.scala:61)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:94)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:623)
	at java.base/java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1136)
	at java.base/java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:635)
	at java.base/java.lang.Thread.run(Thread.java:833)

24/10/07 09:57:13 WARN TaskSetManager: Lost task 4.0 in stage 489.0 (TID 7735) (slave01 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/

Py4JJavaError: An error occurred while calling o6372.confusionMatrix.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 489.0 failed 1 times, most recent failure: Lost task 8.0 in stage 489.0 (TID 7739) (slave01 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)

In [17]:
unenc_test_data = unencode(test_data)

testpred = bestModel.transform(unenc_test_data)
testpred.select("Cover_Type", "pred", "probability").show(10, truncate = False)

24/10/07 09:57:19 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB


+----------+----+---------------------------------------------------------------+
|Cover_Type|pred|probability                                                    |
+----------+----+---------------------------------------------------------------+
|6         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]                              |
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]                              |
|6         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]                              |
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]                              |
|3         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]                              |
|3         |6.0 |[0.0,0.0,0.0,0.3333333333333333,0.0,0.0,0.6666666666666666,0.0]|
|3         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]                              |
|3         |3.0 |[0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0]                              |
|6         |6.0 |[0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0]                              |
|6         |6.0 

In [18]:
evaluate_model(testpred, "Test Data")

24/10/07 09:57:21 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:23 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/context.py:158: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(
24/10/07 09:57:24 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:24 WARN DAGScheduler: Broadcasting large task binary with size 4.5 MiB
24/10/07 09:57:25 ERROR Executor: Exception in task 8.0 in stage 497.0 (TID 7831)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  

24/10/07 09:57:25 WARN TaskSetManager: Lost task 17.0 in stage 497.0 (TID 7840) (slave01 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 8 in stage 497.0 failed 1 times, most recent failure: Lost task 8.0 in stage 497.0 (TID 7831) (slave01 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrappe

Py4JJavaError: An error occurred while calling o6644.confusionMatrix.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 8 in stage 497.0 failed 1 times, most recent failure: Lost task 8.0 in stage 497.0 (TID 7831) (slave01 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrapper
    return f(*args, **kwargs)

r
    return f(*args, **kwargs)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/session.py", line 1459, in prepare
    verify_func(obj)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 2187, in verify
    verify_value(obj)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 2160, in verify_struct
    verifier(v)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 2187, in verify
    verify_value(obj)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 2181, in verify_default
    verify_acceptable_types(obj)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/sql/types.py", line 2006, in verify_acceptable_types
    raise PySparkTypeError(
pyspark.errors.exceptions.base.PySparkTypeError: [CANNOT_ACCEPT_OBJECT_IN_TYPE] `DoubleType()` can not accept object `3` in type `int`.

	at org.apache.spark.api.python.BasePythonRunner$Reade

24/10/07 09:57:25 WARN TaskSetManager: Lost task 14.0 in stage 497.0 (TID 7837) (slave01 executor driver): TaskKilled (Stage cancelled: Job aborted due to stage failure: Task 8 in stage 497.0 failed 1 times, most recent failure: Lost task 8.0 in stage 497.0 (TID 7831) (slave01 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/home/lplab/.local/lib/python3.10/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 83, in wrappe

In [ ]:
spark.stop()